# 2. Extracción de características

En esta sección vamos a presentar las características que podemos extraer tanto del dominio temporal como del dominio frecuencial de un audio. En base a estas características daremos una intuición de cómo se pueden distinguir géneros musicales a nivel exploratorio/cualitativo, antes de proceder a la próxima sección donde utilizaremos modelos de aprendizaje.

## Importaciones

In [ ]:
import librosa

import numpy as np

import matplotlib.pyplot as plt

import IPython.display as ipd
import pandas as pd
import os

___

## 2.1. Explicación de las características

Cuando analizamos una señal de audio, podemos hacerlo desde dos perspectivas principales: el dominio temporal y el dominio frecuencial. Cada uno de estos dominios nos brinda diferentes tipos de información sobre el audio.

### Análisis de una señal de audio: dominio temporal, dominio frecuencial y dominio tempofrecuencial

Hagamos un breve repaso sobre los distintos dominios en los que podemos analizar una señal.

**Dominio temporal: tiempo vs amplitud**

Cuando analizamos el dominio temporal de una señal estamos analizando cómo varía la amplitud de la señal de audio a lo largo del tiempo ("la intensidad de la señal en cada momento"). En este caso:
* **Eje X - tiempo**. El eje horizontal representa el avance temporal medido en segundos.
* **Eje Y - amplitud**. El eje vertical representa la amplitud de la señal, es decir, la desviación de la onda con respecto al eje central. En el dominio temporal se mide como un valor sin dimensiones (adimensional) que oscila entre -1 y 1 en formatos normalizados o como una señal de número entero (por ejemplo, entre -32768 y 32767 señales para 16-bit) en formatos no normalizados. Representa la fuerza o intensidad de la señal.

**Dominio frecuencial: frecuencia vs magnitud**

Cuando analizamos el dominio frecuencial de una señal, estamos analizando la magnitud de cada frencuencia de la señal. En este caso:

* **Eje X - frecuencia.** El eje horizontal representa las frecuencias presentes en la señal, medidas en Hertz (Hz). 
* **Eje Y - magnitud/amplitud.** El eje vertical representa la magnitud de cada frecuencia. Es el módulo del número complejo que representa la contribución de cada frecuencia a la señal global.

Este gráfico representa de forma clara la dicotomía entre ambos dominios.

<img src="img/NB2_timeFreqDomain.png" width="600"/>

*Figura: Dominio temporal vs dominio frecuencial. Fuente: [[1]](https://studyelectrical.com/2023/05/time-domain-analysis-vs-frequency-domain-analysis.html)*

Veamos a continuación un ejemplo práctico utilizando un audio de ejemplo del corpus CCMUSIC:

In [ ]:
file_ejemplo = "ccmusic/train/audios/audio_train_653.wav"
audio, sr = librosa.load(file_ejemplo, sr=None, mono=True)

# Reproducir el audio
ipd.display(ipd.Audio(audio, rate=sr))

# Dominio temporal
plt.figure(figsize=(10, 3))
librosa.display.waveshow(audio, sr=sr, color="#f44", alpha=0.8)
plt.title("Representación tiempo-amplitud de la señal (Forma de onda/oscilograma)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Amplitud")
plt.tight_layout()
plt.show()

# Dominio frecuencial
fft = np.fft.fft(audio)
frequencies = np.fft.fftfreq(len(fft), 1/sr) # *** Explicación abajo ***
magnitude = np.abs(fft)

plt.figure(figsize=(10, 3))
plt.plot(frequencies[:len(frequencies)//2], magnitude[:len(frequencies)//2], color="#4f4") # Representamos solo la mitad dado que el gráfico es simétrico
plt.title("Representación frecuencia-magnitud de la señal (Transformada de fourier)")
plt.xlabel("Frecuencia (Hz)")
plt.ylabel("Magnitud")
plt.tight_layout()
plt.show()

# Frecuencia + tipo
D = np.abs(librosa.stft(audio))
plt.figure(figsize=(10, 3))
librosa.display.specshow(librosa.amplitude_to_db(D, ref=np.max), sr=sr, x_axis='time', y_axis='log')
plt.title("Representación tiempo-frecuencia de la señal (Espectograma)")
plt.xlabel("Tiempo (s)")
plt.ylabel("Frecuencia (Hz)")
plt.tight_layout()
plt.show()

# *** Explicación ***
# np.fft.fftfreq genera un array de frecuencias que corresponde a los componentes de la transformada de Fourier.
# El primer argumento es la longitud de la transformada (igual a la longitud de la señal original).
# El segundo argumento, 1/sr, es el intervalo de tiempo entre muestras, que es el inverso de la tasa de muestreo (sr).
# Esto produce un array de frecuencias que se corresponde con los índices del resultado de np.fft.fft, permitiendo
# visualizar el espectro de frecuencias en Hertz.# 

### Segmentación en bloques

El procesamiento de señales en bloques, o frames, es fundamental en el análisis de audio y otros tipos de señales temporales. Esta técnica permite simplificar y y mejorar en eficiencia el análisis mediante la descomposición de la señal en segmentos manejables. Además, el solapamiento entre bloques ayuda a evitar discontinuidades en la información procesada, mejorando la calidad y la continuidad del análisis. Los parámetros comunes para este proceso son:

- **Tamaño de Frame**: Es el número de muestras que contiene cada bloque. Usualmente, este tamaño es un múltiplo de 2 debido a optimizaciones en la transformada de Fourier.
- **Separación HOP**: Es el intervalo de muestras entre el inicio de un frame y el inicio del siguiente. Si HOP es menor que el tamaño del FRAME, entonces los frames se solapan, lo que puede ayudar a mejorar la continuidad y calidad del análisis.


### Temporización de bloques (frames)

Considerando una señal con muestras $s_i$ ($i=0,..,N-1$) recogidas a una frecuencia de muestreo $sr$, los instantes temporales de cada muestra se calculan como $i\cdot 1/sr$. Al segmentar esta señal en bloques de tamaño $F \geq 1$ y con una separación $H \leq F$, podemos describir el proceso de temporización de los bloques:

- **Inicio de cada bloque**: El instante de tiempo para el comienzo del primer frame de un bloque $k$-ésimo se determina como $H\cdot k/sr$.

Los parámetros $F$ (tamaño del frame) y $H$ (separación entre frames) tienen un impacto directo en cómo se realiza la segmentación:

- **Solapamiento**: Si $H < F$, existe un solapamiento entre frames consecutivos, lo que ayuda a preservar la continuidad entre los segmentos analizados.
- **Número total de bloques ($T$)**: Depende de cómo se quiera gestionar el final de la señal:
    - Si definimos $T = \lfloor \frac{N-F}{H} + 1 \rfloor$, aseguramos que cada bloque, incluido el último, tenga exactamente $F$ muestras, aunque esto podría dejar algunas muestras al final de la señal sin incluir.
    - Si optamos por $T = \lfloor \frac{N}{H} \rfloor$, el último bloque podría no estar completo. Una solución común es rellenar este último bloque con muestras adicionales para completarlo.

### Presentando audios de ejemplo

Para esta sección y la siguiente sección utilizaremos estos audios del corpus CCMUSIC como ejemplo ilustrativo de la explicación de las características.

In [ ]:
# Carga de datos de anotaciones y selección de un audio por jerarquía

# Jerarquía 1: Classic vs Non_classic

anotaciones_jerarquia1 = pd.read_csv('ccmusic/train/annotations.csv')
seleccionados_jerarquia1 = [group.sample(1) for _, group in anotaciones_jerarquia1.groupby('label_name')]
seleccionados_jerarquia1 = pd.concat(seleccionados_jerarquia1).reset_index(drop=True)
audios_ejemplo_jerarquia1 = [
    (fila['audio_file'], fila['label_name']) for _, fila in seleccionados_jerarquia1.iterrows()
]
print("Audios de ejemplo de la jerarquía fst_level_label \n", audios_ejemplo_jerarquia1)

for file, label in audios_ejemplo_jerarquia1:
    print(f"{label}: {file}")
    display(ipd.Audio(file, autoplay=True))

In [ ]:
# Jerarquía 2: Symphony, opera, solo, chamber, pop, dance, indie, soul, rock

anotaciones_jerarquia2 = pd.read_csv('ccmusic2/train/annotations.csv')
seleccionados_jerarquia2 = [group.sample(1) for _, group in anotaciones_jerarquia2.groupby('label_name')]
seleccionados_jerarquia2 = pd.concat(seleccionados_jerarquia2).reset_index(drop=True)
audios_ejemplo_jerarquia2 = [
    (fila['audio_file'], fila['label_name']) for _, fila in seleccionados_jerarquia2.iterrows()
]
print("Audios de ejemplo de la jerarquía snd_level_label\n", audios_ejemplo_jerarquia2)

for file, label in audios_ejemplo_jerarquia2:
    print(f"{label}: {file}")
    display(ipd.Audio(file, autoplay=True))

### Características del dominio temporal

* **Amplitude Envelope (AE)**:
    - Intuitivamente, la envolvente de una señal representa el "borde del oscilograma". Permite visualizar de una forma más intuitiva cómo varía la intensidad de la señal a lo largo del tiempo.
    
    - Matemáticamente, se calcula de la siguiente manera.   
        Vamos a considerar una señal de audio agrupada en $T$ *frames* o bloques temporales $(k=0,...,T-1)$ de tamaño $F$, con desplazamiento o *hop* $H$. Para cada *frame*
        1. Se toma el máximo el máximo valor de amplitud (dando esa impresión de borde del oscilograma en la representación final):
        $$AE_k = \max_{i=kH}^{kH+F - 1} s(i)$$        
        donde $AE_k$ es la envolvente del *frame* $k$, $F$ es el tamaño de *frame*, $H$ es el *hop*, y $s(i)$ es la amplitud de la señal en el índice $i$.

In [ ]:
def amplitude_envelope(signal,frame_size=1024,hop_length=512):
    F=frame_size                                                    # Tamaño de frame
    H=hop_length                                                    # Número de muestras que se desplazan al avanzar de un bloque al siguiente.
    N=signal.shape[0]                                               # Número de muestras en la señal
    return np.array([max(signal[k:k+F]) for k in range(0, N, H)])   # Para cada frame k se calcula el máximo de la amplitud de la señal en ese frame

In [ ]:
FRAMES_SIZE = 1024
HOP_LENGTH = 512
SR = 22050

fig, ax = plt.subplots(1, 2, figsize=(20, 6))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):

    signal, sr = librosa.load(file, sr=None, mono=True)
    ae_signal = amplitude_envelope(signal, FRAMES_SIZE, HOP_LENGTH)
    frames = range(0, len(ae_signal))  
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)

    librosa.display.waveshow(signal, sr=22050, alpha=0.6, color="b", ax=ax[i])
    ax[i].plot(t, ae_signal, color="red", label="AE")
    ax[i].set_title(f"Envolvente de la señal de ejemplo de categoría {label}")
    ax[i].set_xlabel("Tiempo (s)")
    ax[i].set_ylabel("Amplitud")
    ax[i].legend()

plt.tight_layout()
plt.show()

In [ ]:
FRAMES_SIZE = 1024
HOP_LENGTH = 512
SR = 22050

fig, ax = plt.subplots(3, 3, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    ae_signal = amplitude_envelope(signal, FRAMES_SIZE, HOP_LENGTH)
    frames = range(0, len(ae_signal))  
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)

    row = i // 3
    col = i % 3

    librosa.display.waveshow(signal, sr=22050, alpha=0.6, color="b", ax=ax[row, col])
    ax[row, col].plot(t, ae_signal, color="red", label="AE")
    ax[row, col].set_title(f"Envolvente de la señal de ejemplo de categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("Amplitud")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **RMS (Root Mean Square)**:

    - La RMS (Root Mean Square) de una señal sirve para estimar la energía de la señal en distintos puntos del tiempo, lo que puede ayudar a detectar silencios y la dinámica de la señal. 
    
    - Para calcular el RMS de una señal segmentada en $T$ bloques ($k=1,..,T-1$) de tamaño $F$, matemáticamente:  
    $$RMS_k=\sqrt{\frac{1}{F} \cdot \sum_{i=k \cdot F}^{(k+1)\cdot F-1}{s(i)^2}}$$  
    donde $S(i)$ representa la amplitud de la señal en el instante i.


In [ ]:
def calculate_rms(signal, frame_size=1024, hop_length=512):
    rms = librosa.feature.rms(y=signal, frame_length=frame_size, hop_length=hop_length)[0]
    return rms

In [ ]:
FRAMES_SIZE = 1024
HOP_LENGTH = 512
SR = 22050

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):

    signal, sr = librosa.load(file, sr=None, mono=True)
    rms_signal = calculate_rms(signal, FRAMES_SIZE, HOP_LENGTH)
    frames = range(0, len(rms_signal))  
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # BER
    ax[1, i].plot(t, rms_signal, color="red", label="RMS")
    ax[1, i].set_title(f"Root Mean Square de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("RMS")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")
# plt.tight_layout()
# plt.show()

fig, ax = plt.subplots(3, 3, figsize=(20, 15))
for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    ber_spec = calculate_rms(signal, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(ber_spec))
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, ber_spec, color="red", label="RMS")
    ax[row, col].set_title(f"Root Mean Square de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("RMS")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **ZCR (Zero Crossing Rate)**:

    - La ZCR mide cuantas veces la señal de audio cruza el eje horizontal respecto a la longitud total de la señal, es decir, cuantas veces la amplitud pasa de negativa a positiva o viceversa.

    - Para calcular la ZCR de una señal segmentada en $T$ bloques ($k=1,..,T-1$) de tamaño $F$, matemáticamente:
        $$ZCR_k=\sum_{i=k \cdot F}^{(k+1)\cdot F-1} \frac{1}{2} | \text{sgn($s(i)$)- sgn($s(i+1)$)}  |$$
        donde $S(i)$ es la amplitud de la señal en el instante $i$ y la funcion signo se define como
        $$ \text{sgn($z$)}=\begin{Bmatrix} 
        1  &  z>0 \\
        0  &  z=0 \\
        -1 &  z<0
        \end{Bmatrix}$$
        El resultado para cada bloque puede ser normalizado $ZCR_k/F$ para que sus valores estén entre [0,1]

In [ ]:
def calculate_zcr(signal, frame_size=1024, hop_length=512):
    zcr = librosa.feature.zero_crossing_rate(y=signal, frame_length=frame_size, hop_length=hop_length)[0]
    return zcr

In [ ]:
FRAMES_SIZE = 1024
HOP_LENGTH = 512
SR = 22050

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):

    signal, sr = librosa.load(file, sr=None, mono=True)
    rms_signal = calculate_zcr(signal, FRAMES_SIZE, HOP_LENGTH)
    frames = range(0, len(rms_signal))  
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # BER
    ax[1, i].plot(t, rms_signal, color="red", label="ZCR")
    ax[1, i].set_title(f"Zero Crossing Rate de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("ZCR")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")
# plt.tight_layout()
# plt.show()

fig, ax = plt.subplots(3, 3, figsize=(20, 15))
for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    ber_spec = calculate_zcr(signal, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(ber_spec))
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, ber_spec, color="red", label="ZCR")
    ax[row, col].set_title(f"Zero Crossing Rate de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("ZCR")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

### Características del dominio frecuencial

* **Band Energy Ratio (BER)**
    - Intuitivamente, el BER (relación de energía entre bandas) compara cuánta energía hay en los sonidos graves en comparación con los sonidos agudos de una señal (compara la energía acumulada en las frecuencias bajas frente a las frecuencias altas).

    - Es útil para distinguir audios conversacionales de audios musicales y también para distinguir entre distintos estilos de música.

    - Matemáticamente, se calcula de la siguiente forma.   
        Vamos a considerar una señal dividida en $T$ bloques temporales o *frames* $(k=0,...,T-1)$. Para cada *frame* podemos calcular su $BER_k$. Sea $F$ el umbral considerado, por encima del cuál se consideran frecuencias altas y por debajo las frecuencias bajas. Para cada *frame*:  
        
        1. Calculamos la energía de la banda baja: $\text{Energía banda baja}_k = \sum_{n=0}^{F-1} m_k(n)^2$  
        2. Calculamos la energía de la banda alta: $\text{Energía banda alta}_k = \sum_{n=F}^{N} m_k(n)^2$  
        3. Tomamos el cociente para obtener el $BER$ del *frame* k:  
            $$\text{BER}_k = \frac{\text{Energía banda baja}_k}{\text{Energía banda alta}_k} = \frac{\sum_{n=0}^{F-1} m_k(n)^2}{\sum_{n=F}^{N} m_k(n)^2}$$

        donde $F$ representa la frecuencia umbral de separación de franjas frecuenciales altas y bajas, $k$ representa el *frame* y $m_k(n)$ es el valor de la magnitud de la frecuencia n en el *frame* k.


In [ ]:
def calculate_ber(signal, split_freq, sample_rate, frame_size=1024, hop_length=512):

    # 1. Cálculo del espectograma
    spec = librosa.stft(signal, n_fft=frame_size, hop_length=hop_length)        # Calcular el espectograma de la señal
    modified_spec = np.abs(spec).T                                              # Convertir las magnitudes de números complejos a valor absoluto

    # 2. Determinar los límites de las bandas de frecuencia
    range_of_freq = sample_rate / 2
    change_per_bin = range_of_freq / spec.shape[0]
    split_freq_bin = int(np.floor(split_freq / change_per_bin))

    # 3. Calcular el BER para cada frame
    res = []
    for sub_arr in modified_spec:

        # Densidad de energía para las frecuencias bajas
        low_freq_density = sum(i ** 2 for i in sub_arr[:split_freq_bin])
        # Densidad de energía para las frecuencias altas
        high_freq_density = sum(i ** 2 for i in sub_arr[split_freq_bin:])
        high_freq_density = high_freq_density if high_freq_density > 0 else 1e-10

        # Calcular cociente
        ber_val = low_freq_density / high_freq_density
        res.append(ber_val)

    return np.array(res)


In [ ]:
SPLIT_FREQ = 200
SR = 22050
FRAMES_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):

    signal, sr = librosa.load(file, sr=None, mono=True)
    ber_spec = calculate_ber(signal, SPLIT_FREQ, SR, FRAMES_SIZE, HOP_LENGTH)
    frames = range(0, len(ber_spec))
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # BER
    ax[1, i].plot(t, ber_spec, color="red", label="BER")
    ax[1, i].set_title(f"BER de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("BER")
    ax[1, i].legend()

plt.tight_layout()
plt.show()


In [ ]:
SPLIT_FREQ = 200
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")
# plt.tight_layout()
# plt.show()

fig, ax = plt.subplots(3, 3, figsize=(20, 15))
for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    ber_spec = calculate_ber(signal, SPLIT_FREQ, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(ber_spec))
    t = librosa.frames_to_time(frames, hop_length=HOP_LENGTH, sr=SR)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, ber_spec, color="red", label="BER")
    ax[row, col].set_title(f"BER de la señal de categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("BER")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **Spectral Centroid (SC)**

    - Intuitivamente, el centroide espectral representa el "centro de gravedad" de la distribución de frecuencias, es decir, la banda de frecuencias en torno a la cual se concentra la mayor parte de la energía de la señal. 
    
    - Es útil para distinguir sonidos con distinto "brillo". Si el centroide espectral es alto, el sonido tiende a ser más brillante o agudo, mientras que si es bajo, el sonido tiende a ser más oscuro o grave.

    - Matemáticamente, se calcula de la siguiente forma. Vamos a considerar una señal dividida en $T$ bloques temporales o *frames* $(k=0,\ldots,T-1)$. Para cada *frame* podemos calcular su $SC_k$. Para cada *frame*:

        1. Calculamos el numerador como la suma ponderada de las magnitudes de las frecuencias: $sum_{n=0}^{N} n \cdot m_k(n)$
        2. Calculamos el denominador como la suma de las magnitudes de las frecuencias: $\sum_{n=0}^{N} m_k(n)$
        3. Tomamos cociente: 
        $$\text{SC}_k = \frac{\sum_{n=0}^{N} n \cdot m_k(n)}{\sum_{n=0}^{N} m_k(n)}$$
        donde $k$ representa el *frame* y $m_k(n)$ es el valor de la magnitud de la frecuencia $n$ en el *frame* $k$.

        Esta ecuación se corresponde con la ecuación matemática de la esperanza/media estadística de una distribución, en este caso, la distribución de frecuencias.

In [ ]:
def calculate_spectral_centroid(signal, sample_rate, frame_size=1024, hop_length=512):
    spec_centroid = librosa.feature.spectral_centroid(y=signal, 
                                                      sr=sample_rate, 
                                                      n_fft=frame_size, 
                                                      hop_length=hop_length)[0]
    return spec_centroid

In [ ]:
SPLIT_FREQ = 200
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))
for i, (signal, label) in enumerate(audios_ejemplo_jerarquia1):
    sc_spec = calculate_spectral_centroid(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(sc_spec))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # Spectral Centroid
    ax[1, i].plot(t, sc_spec, color="red", label="SC")
    ax[1, i].set_title(f"SC de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("SC")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))

# Mostrar oscilogramas
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")

# plt.tight_layout()
# plt.show()

# Mostrar centroides espectrales
fig, ax = plt.subplots(3, 3, figsize=(20, 15))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    sc_spec = calculate_spectral_centroid(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(sc_spec))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, sc_spec, color="red", label="SC")
    ax[row, col].set_title(f"SC de la señal de ejemplo de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("SC")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **Spectral Bandwidth**
    - Intuitivamente, el ancho de banda espectral representa la dispersión o la extensión de la distribución de frecuencias. Si el ancho de banda es alto, las frecuencias de la señal están más dispersas, mientras que si es bajo, las frecuencias están más concentradas en torno a un punto central.

    - Es útil para distinguir entre sonidos con diferentes "texturas".

    - Matemáticamente, se calcula de la siguiente forma. Vamos a considerar una señal dividida en $T$ bloques temporales o *frames* $(k=0,\ldots,T-1)$. Para cada *frame* podemos calcular su $SBW_k$. Para cada *frame*:

        1. Calculamos el centroide espectral (SC) para el *frame* k: $\text{SC}_k = \frac{\sum_{n=0}^{N} n \cdot m_k(n)}{\sum_{n=0}^{N} m_k(n)}$
        2. Calculamos el numerador como la suma ponderada de las diferencias cuadradas de las magnitudes de las frecuencias con respecto al centroide: $\sum_{n=0}^{N} (n - \text{SC}_k)^2 \cdot m_k(n)$
        3. Calculamos el denominador como la suma de las magnitudes de las frecuencias: $\sum_{n=0}^{N} m_k(n)$
        4. Tomamos la raíz cuadrada del cociente para obtener el $SBW$ del *frame* $k$: $\text{SBW}_k = \sqrt{\frac{\sum_{n=0}^{N} (n - \text{SC}_k)^2 \cdot m_k(n)}{\sum_{n=0}^{N} m_k(n)}}$

        donde $k$ representa el *frame* y $m_k(n)$ es el valor de la magnitud de la frecuencia $n$ en el *frame* $k$.

        Esta ecuación se corresponde con la ecuación matemática de la desviación estándar de una distribución, en este caso, la distribución de frecuencias.

In [ ]:
def calculate_spectral_bandwidth(signal, sample_rate, frame_size=1024, hop_length=512):
    spec_bandwidth = librosa.feature.spectral_bandwidth(y=signal, sr=sample_rate, n_fft=frame_size, hop_length=hop_length)[0]
    return spec_bandwidth

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):
    
    signal, sr = librosa.load(file, sr=None, mono=True)
    sbw_spec = calculate_spectral_bandwidth(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(sbw_spec))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # Spectral Bandwidth
    ax[1, i].plot(t, sbw_spec, color="red", label="SBW")
    ax[1, i].set_title(f"SBW de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("SBW")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))

# # Mostrar oscilogramas
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")

# plt.tight_layout()
# plt.show()

# Mostrar espectrales
fig, ax = plt.subplots(3, 3, figsize=(20, 15))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    sbw_spec = calculate_spectral_bandwidth(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(sbw_spec))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, sbw_spec, color="red", label="SBW")
    ax[row, col].set_title(f"SBW de la señal de ejemplo de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("SBW")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **Chroma STFT**:
    - El Chroma STFT (Short-Time Fourier Transform) captura la esencia de las clases de tonos en la música, destacando la importancia de las dimensiones armónicas y melódicas. La utilización de la transformada de Fourier de tiempo corto permite analizar la señal en segmentos temporales, adecuados para captar variaciones a lo largo del tiempo. A continuación se detalla su cálculo:


    - **Transformada de Fourier de Tiempo Corto**: Para cada segmento o *frame* de la señal:
        1. Calculamos la transformada de Fourier de tiempo corto para obtener la representación en frecuencia del *frame* $k$: 
            $$
            X(k, \omega) = \sum_{n=0}^{N-1} s(n + kH) \cdot w(n) \cdot e^{-j \omega n}
            $$
            donde $s(i)$ es la amplitud de la señal en el instante $i$, $k$ es el índice del *frame*, $H$ es el tamaño del salto entre *frames*, $w(n)$ es la ventana de análisis, $N$ es el número de puntos en la FFT, y $\omega$ es la frecuencia angular.
        2. Mapeamos la magnitud del espectro $|X(k, \omega)|$ a las 12 clases cromáticas, sumando la energía de las frecuencias que caen dentro de cada clase tonal. El mapeo se hace de acuerdo a la afinación temperada, que divide la octava en 12 tonos de igual relación de frecuencia. El resultado se normaliza y se representa como un vector de 12 componentes que refleja la fuerza relativa de cada tono en el *frame*:
            $$
            C(k, m) = \sum_{\omega \in \text{bin}(m)} |X(k, \omega)|
            $$
            donde $m$ es el índice de una de las 12 clases cromáticas y $\text{bin}(m)$ incluye los índices de frecuencia que corresponden a la clase tonal $m$ en la escala temperada. 


In [ ]:
def calculate_chroma_stft(signal, sample_rate, frame_size=1024, hop_length=512):
    chroma_stft = librosa.feature.chroma_stft(y=signal, sr=sample_rate, n_fft=frame_size, hop_length=hop_length)
    # Esto devuelve una matriz donde las columnas son los frames y las filas son los 12 bins de chroma
    return chroma_stft

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):
    
    signal, sr = librosa.load(file, sr=None, mono=True)
    chroma = calculate_chroma_stft(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(chroma))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # Spectral Chroma STFT
    img = librosa.display.specshow(chroma, x_axis='time', y_axis='chroma', hop_length=HOP_LENGTH, sr=sr, ax=ax[1, i])
    fig.colorbar(img, ax=ax[1, i])
    ax[1, i].set_title(f"Chroma STFT de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("Clase")

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))

# # Mostrar oscilogramas
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")

# plt.tight_layout()
# plt.show()

# Mostrar espectrales
fig, ax = plt.subplots(3, 3, figsize=(20, 15))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    chroma = calculate_chroma_stft(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(chroma))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)
    row = i // 3
    col = i % 3
    img = librosa.display.specshow(chroma, x_axis='time', y_axis='chroma', hop_length=HOP_LENGTH, sr=sr, ax=ax[row, col])
    fig.colorbar(img, ax=ax[row, col])
    ax[row, col].set_title(f"Chroma STFT de la señal de categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("Clase")


plt.tight_layout()
plt.show()

* **Spectral Rolloff**:  
  
    - El Spectral Rolloff es una medida utilizada para determinar el límite superior de las frecuencias en una señal de audio, reflejando el punto por debajo del cual se encuentra un porcentaje determinado de la energía espectral total. La técnica implica:

    - **Transformada de Fourier de Tiempo Corto**: Similar al cálculo de Chroma STFT, para cada *frame*:
        1. Realizamos una transformada de Fourier de tiempo corto para cada *frame* $k$:
            $$
            X(k, \omega) = \sum_{n=0}^{N-1} s(n + kH) \cdot w(n) \cdot e^{-j \omega n}
            $$
        2. Calculamos el rolloff espectral para cada *frame* como la frecuencia mínima para la cual la suma acumulada de la magnitud del espectro excede el 85% (o un porcentaje especificado) del total de la energía espectral en ese *frame*:
            $$
            R(k) = \min \left( \omega : \sum_{i=0}^{\omega} |X(k, i)| \geq 0.85 \times \sum_{i=0}^{N-1} |X(k, i)| \right)
            $$

In [ ]:
def calculate_spectral_rolloff(signal, sample_rate, frame_size=1024, hop_length=512):
    rolloff = librosa.feature.spectral_rolloff(y=signal, sr=sample_rate, n_fft=frame_size, hop_length=hop_length)[0]
    return rolloff

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):
    
    signal, sr = librosa.load(file, sr=None, mono=True)
    rolloff = calculate_spectral_rolloff(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(rolloff))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # Spectral Rolloff
    ax[1, i].plot(t, rolloff, color="red", label="Rolloff")
    ax[1, i].set_title(f"Rolloff de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("Rolloff")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))

# # Mostrar oscilogramas
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")

# plt.tight_layout()
# plt.show()

# Mostrar espectrales
fig, ax = plt.subplots(3, 3, figsize=(20, 15))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    rolloff = calculate_spectral_rolloff(signal, SR, FRAME_SIZE, HOP_LENGTH)
    frames = range(0, len(rolloff))
    t = librosa.frames_to_time(frames, sr=SR, hop_length=HOP_LENGTH)
    row = i // 3
    col = i % 3
    ax[row, col].plot(t, rolloff, color="red", label="Rolloff")
    ax[row, col].set_title(f"Rolloff de la señal de ejemplo de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("Rolloff")
    ax[row, col].legend()

plt.tight_layout()
plt.show()

* **Mel Frequency Cepstral Coefficients (MFCC)**:  

    - Los coeficientes de Cepstrum de Frecuencia Mel (MFCC) cpturan las características espectrales de la señal basándose en la percepción auditiva humana y no simplemente en las frecuencias físicas.

    - **Extracción de MFCC**: Para extraer los MFCC de una señal de audio, se sigue el siguiente proceso dividido en varias etapas:
        1. **Segmentación en Frames**: Dividimos la señal en varios segmentos cortos o *frames*, usando una ventana de análisis para minimizar las discontinuidades en los bordes de los *frames* de la siguiente manera:
            $$
            s_k(n) = s(n + kH) \cdot w(n)
            $$
            donde $s_k(n)$ es la muestra $n$ en el *frame* $k$, $s(n)$ es la muestra original, $kH$ es el desplazamiento del inicio del *frame* $k$ en muestras, y $w(n)$ es la función ventana aplicada.
        2. **Transformada de Fourier**: Aplicamos la Transformada de Fourier de tiempo corto (STFT) para transformar cada *frame* del dominio del tiempo al dominio de la frecuencia:
            $$
            X(k, \omega) = \sum_{n=0}^{N-1} s_k(n) \cdot e^{-j \omega n}
            $$
            donde $X(k, \omega)$ es el espectro del *frame* $k$ y $\omega$ representa las frecuencias analizadas.
        3. **Filtrado en la Escala Mel**: Utilizamos bancos de filtros Mel para capturar las características relevantes según la percepción auditiva humana. Estos filtros están distribuidos logarítmicamente para imitar la percepción del oído humano:
            $$
            M(k, m) = \sum_{\omega=0}^{N-1} |X(k, \omega)| \cdot H_m(\omega)
            $$
            donde $M(k, m)$ es la energía del *frame* $k$ en el filtro Mel $m$, y $H_m(\omega)$ es el filtro Mel aplicado a la frecuencia $\omega$.
        4. **Cálculo de los Coeficientes Cepstrales**: Calculamos el logaritmo de cada energía de banda Mel, y luego aplicamos la Transformada Discreta de Coseno (DCT) para obtener los coeficientes cepstrales que son los MFCC:
            $$
            MFCC(k, c) = \text{DCT} \left( \log(M(k, m)) \right) = \sum_{m=0}^{M-1} \log(M(k, m)) \cdot \cos\left(\frac{\pi c (2m+1)}{2M}\right)
            $$
            donde $MFCC(k, c)$ son los coeficientes MFCC del *frame* $k$ para el coeficiente $c$, y $M$ es el número total de filtros Mel.


In [ ]:
def calculate_mfccs(signal, sample_rate, n_mfcc=13, frame_size=1024, hop_length=512):
    mfcc = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=n_mfcc, n_fft=frame_size, hop_length=hop_length)
    # Esto devuelve una matriz donde las columnas son los frames y las filas son los coeficientes MFCC
    return mfcc

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

fig, ax = plt.subplots(2, 2, figsize=(20, 10))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia1):
    
    signal, sr = librosa.load(file, sr=None, mono=True)
    mfccs = calculate_mfccs(signal, sr, n_mfcc=13, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH)
    mfcc_1 = mfccs[0]  # Primera característica de MFCC
    frames = range(len(mfcc_1))
    t = librosa.frames_to_time(frames, sr=sr, hop_length=HOP_LENGTH)

    # Oscillogram
    librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[0, i])
    ax[0, i].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
    ax[0, i].set_xlabel("Tiempo (s)")
    ax[0, i].set_ylabel("Amplitud")

    # Spectral Rolloff
    ax[1, i].plot(t, mfcc_1, label="MFCC 1")
    ax[1, i].set_title(f"Caracerística 1 de MFCC de la señal de ejemplo de la categoría {label}")
    ax[1, i].set_xlabel("Tiempo (s)")
    ax[1, i].set_ylabel("Caracerística 1 de MFCC")
    ax[1, i].legend()

plt.tight_layout()
plt.show()

In [ ]:
SR = 22050
FRAME_SIZE = 1024
HOP_LENGTH = 512

# fig, ax = plt.subplots(3, 3, figsize=(20, 15))

# # Mostrar oscilogramas
# for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
#     signal, sr = librosa.load(file, sr=None, mono=True)
#     row = i // 3
#     col = i % 3
#     librosa.display.waveshow(signal, sr=SR, alpha=0.8, color="b", ax=ax[row, col])
#     ax[row, col].set_title(f"Oscilograma de la señal de ejemplo de la categoría {label}")
#     ax[row, col].set_xlabel("Tiempo (s)")
#     ax[row, col].set_ylabel("Amplitud")

# plt.tight_layout()
# plt.show()

# Mostrar espectrales
fig, ax = plt.subplots(3, 3, figsize=(20, 15))

for i, (file, label) in enumerate(audios_ejemplo_jerarquia2):
    signal, sr = librosa.load(file, sr=None, mono=True)
    mfccs = calculate_mfccs(signal, sr, n_mfcc=13, frame_size=FRAME_SIZE, hop_length=HOP_LENGTH)
    mfcc_1 = mfccs[0]  # Primera característica de MFCC
    frames = range(len(mfcc_1))
    t = librosa.frames_to_time(frames, sr=sr, hop_length=HOP_LENGTH)
    row = i // 3
    col = i % 3

    ax[row, col].plot(t, mfcc_1, color="red", label="MFCC 1")
    ax[row, col].set_title(f"Caracerística 1 de MFCC de la señal de ejemplo de la categoría {label}")
    ax[row, col].set_xlabel("Tiempo (s)")
    ax[row, col].set_ylabel("Caracerística 1 de MFCC")
    ax[row, col].legend()
plt.tight_layout()
plt.show()

____

## 2.2. Caso práctico: las características distinguen géneros muscicales

____

## 2.3. Extracción de características para el dataset CCMUSIC

En la actualidad es muy común el uso de espectogramas junto con redes convolucionales CNN para realizar clasificación de audio. No obstante, tradicionalmente, para entrenar modelos clásicos de Machine Learning se utilizaba extracción de características.

Al igual que en el caso del análisis de textos se utilizada el TfIdf junto con SVM ("bag of words"), en el análisis de audio se identificaba cada audio con una serie de características y se construía un dataset tabular (de ahí que hayamos titulado este proyecto "bag of songs").

En esta sección, vamos a extraer las características anteriores para cada audio y vamos a construir un dataset tabular que nos permita clasificar utilizando modelos clásicos y datos tabulares.

In [ ]:
def store_features(audio_path, label, csv_file):
    audio_data, sr = librosa.load(audio_path, sr=None)
    
    envelope = amplitude_envelope(audio_data)
    rms = calculate_rms(audio_data)
    zcr = calculate_zcr(audio_data)
    ber = calculate_ber(audio_data, 500, sr)
    
    spec_cent = calculate_spectral_centroid(audio_data, sr)
    spec_bw = calculate_spectral_bandwidth(audio_data, sr)
    chroma_stft = calculate_chroma_stft(audio_data, sr)
    rolloff = calculate_spectral_rolloff(audio_data, sr)
    
    mfcc = calculate_mfccs(audio_data, sr, n_mfcc=13)

    features = [np.mean(envelope), np.mean(rms), np.mean(zcr), np.mean(ber), np.mean(spec_cent),
                np.mean(spec_bw), np.mean(chroma_stft), np.mean(rolloff)] + np.mean(mfcc, axis=1).tolist()
    
    # Writing to CSV
    with open(csv_file, 'a') as f:
        row = f"{audio_path},{label}," + ",".join(map(str, features))
        f.write(row + "\n")

In [ ]:
def process_corpus(corpus_folder):
    # Setup folder structure
    partitions = ['train', 'validation', 'test']
    for part in partitions:
        annotations_path = f"{corpus_folder}/{part}/annotations.csv"
        features_path = f"{corpus_folder}/{part}/features.csv"
        
        # Prepare CSV for features
        with open(features_path, 'w') as f:
            headers = ["audio_file", "label", "mean_envelope", "mean_rms", "mean_zcr",
                       "mean_ber", "mean_spec_cent", "mean_spec_bw", "mean_chroma_stft",
                       "mean_rolloff"] + [f"mean_mfcc{i+1}" for i in range(13)]
            f.write(",".join(headers) + "\n")
        
        # Process each audio file
        annotations = pd.read_csv(annotations_path)
        for index, row in annotations.iterrows():
            audio_file = row['audio_file']
            print(audio_file)
            label = row['label_name']
            store_features(audio_file, label, features_path)

In [ ]:
process_corpus('ccmusic')

In [ ]:
process_corpus('ccmusic2')

____